In [ ]:
import pandas as pd

file_path = "merged_games.csv"  
data = pd.read_csv(file_path)

print(data.head())
print(data.info())

    GAME_ID        DATE  SEASON     TEAM_ID       EFG       FTR  OREB_PCT  \
0  22200477  2022-12-22    2022  1610612740  0.554945  0.296703  0.347826   
1  22200477  2022-12-22    2022  1610612759  0.527778  0.300000  0.318182   
2  22200478  2022-12-22    2022  1610612762  0.581395  0.244186  0.190476   
3  22200478  2022-12-22    2022  1610612764  0.603659  0.207317  0.085714   
4  22200466  2022-12-21    2022  1610612739  0.541176  0.329412  0.214286   

    TOV_PCT   OFF_RTG   DEF_RTG  HOME  MOV  
0  0.141808  1.212938  1.115561     1  9.0  
1  0.143001  1.115561  1.212938     0 -9.0  
2  0.103539  1.221498  1.160862     1  8.0  
3  0.100523  1.160862  1.221498     0 -8.0  
4  0.093179  1.159479  1.074179     1  8.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53102 entries, 0 to 53101
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GAME_ID   53102 non-null  int64  
 1   DATE      53102 non-null  object 
 2

In [ ]:
data['DATE'] = pd.to_datetime(data['DATE'])
data = data.sort_values(by=['TEAM_ID', 'SEASON', 'DATE'])
stats_columns = ['EFG', 'FTR', 'OREB_PCT', 'TOV_PCT', 'OFF_RTG', 'DEF_RTG']
rolling_data = []

for _, group in data.groupby(['TEAM_ID', 'SEASON']):
    rolling_means = group[stats_columns].rolling(window=10, min_periods=10).mean()    
    group = group.reset_index(drop=True)
    for stat in stats_columns:
        group[f'{stat}_ROLLING_MEAN'] = rolling_means[stat]
    
    rolling_data.append(group)

data = pd.concat(rolling_data, ignore_index=True)
data = data.dropna(subset=[f'{stat}_ROLLING_MEAN' for stat in stats_columns]).reset_index(drop=True)
print(data.head())


    GAME_ID       DATE  SEASON     TEAM_ID       EFG       FTR  OREB_PCT  \
0  22200070 2022-10-28    2022  1610612737  0.628866  0.185567  0.263158   
1  22200107 2022-11-02    2022  1610612738  0.467742  0.290323  0.160000   
2  22200008 2022-10-19    2022  1610612739  0.558442  0.298701  0.171429   
3  22200466 2022-12-21    2022  1610612739  0.541176  0.329412  0.214286   
4  12200016 2022-10-04    2022  1610612740  0.553763  0.387097  0.306122   

    TOV_PCT   OFF_RTG   DEF_RTG  HOME   MOV  EFG_ROLLING_MEAN  \
0  0.054093  1.347602  1.134522     1  24.0          0.532308   
1  0.117766  1.019120  1.012793     1  -1.0          0.507733   
2  0.163273  1.070118  1.068461     1  -3.0          0.539690   
3  0.093179  1.159479  1.074179     1   8.0          0.543906   
4  0.168144  1.113605  1.059681     1   4.0          0.546355   

   FTR_ROLLING_MEAN  OREB_PCT_ROLLING_MEAN  TOV_PCT_ROLLING_MEAN  \
0          0.235472               0.235431              0.119620   
1          0.261

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Prepare data
X = data.drop(columns=['MOV', 'GAME_ID', 'DATE', 'SEASON'])  
y = data['MOV']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=13)

model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}, R²: {r2:.2f}")


Epoch 1/50


C:\Users\saraf\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 189.8133 - mae: 11.1166 - val_loss: 255.5227 - val_mae: 14.6555
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 187.7016 - mae: 11.0878 - val_loss: 254.7510 - val_mae: 14.6342
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 182.7581 - mae: 11.0242 - val_loss: 253.9681 - val_mae: 14.6120
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 183.6392 - mae: 10.9927 - val_loss: 253.1425 - val_mae: 14.5883
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 182.4455 - mae: 10.9939 - val_loss: 252.1916 - val_mae: 14.5601
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 182.7484 - mae: 10.9562 - val_loss: 251.3011 - val_mae: 14.5333
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 178.6232 - mae: 10.8165 - val_loss: 250.3719 - val_mae: 14.5062
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 179.2935 - mae: 10.8377 - val_loss: 249.3765 - val_mae: 14.4773
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s